In [6]:
from typing import Any, Type
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool, DuckDuckGoSearchRun
from pydantic import BaseModel, Field
from langchain.agents import initialize_agent, AgentType
from langchain.utilities import DuckDuckGoSearchAPIWrapper, WikipediaAPIWrapper
from langchain.schema import SystemMessage
from langchain.document_loaders import WebBaseLoader

llm = ChatOpenAI(
  temperature=0.1,
)
def save_research_results(filename, research_content):
    try:
        with open(filename, 'w', encoding='utf-8') as file:
            file.write(research_content)
        print(f"Research results successfully saved to {filename}")
    except IOError as e:
        print(f"An error occurred while writing to the file: {e}")

class DuckDuckGoSearchToolArgsSchema(BaseModel):
  query: str = Field(description="The query you will search for")
  url: str = Field(description="The query you will search for")

class DuckDuckGoSearchTool(BaseTool):
  name = "DuckDuckGoSearchTool"
  description = """
  Use this tool when you need to search using the DuckDuckGo search engine.
  DuckDuckGo searches for the given keyword as a query and returns the url of the site that best describes the keyword.
  """
  args_schema: Type[DuckDuckGoSearchToolArgsSchema] = DuckDuckGoSearchToolArgsSchema
  
  def _run(self, query):
    ddg = DuckDuckGoSearchRun()
    loader = WebBaseLoader(
      web_path = url
    )
    result = loader.load()
    print("result>>>>>>>>>>>>>",result)

    return ddg.run(query)
   
class Wikipedia_SearchToolArgsSchema(BaseModel):
  query: str = Field(description="The query you will search for")

class Wikipedia_SearchTool(BaseTool):
  name = "Wikipedia_SearchTool"
  description = """
  Use this tool if you need to find content on Wikipedia related to a topic.
  Example query: Research about the XZ backdoor
  """
  args_schema: Type[Wikipedia_SearchToolArgsSchema] = Wikipedia_SearchToolArgsSchema
  
  def _run(self, query):
    wiki = WikipediaAPIWrapper()
    return wiki.run(query)

agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.OPENAI_FUNCTIONS,
    handle_parsing_errors=True,
    tools=[
      DuckDuckGoSearchTool(),
      Wikipedia_SearchTool(),
    ],
    agent_kwargs={
        "system_message": SystemMessage(
            content="""
           You are a meticulous researcher. When doing research, you have a habit of first consulting Wikipedia and then doing a web search using DuckDuckGo. Among the information found in multiple places, overlapping information is shown only once and information that is not known is not made up. Websites that find relevant content will post the source with it's link.
        """
        )
    },
)
results = agent.invoke("Research about the XZ backdoor.")
save_research_results("search_result.txt", results["output"])




> Entering new AgentExecutor chain...

Invoking: `Wikipedia_SearchTool` with `{'query': 'XZ backdoor'}`


Page: XZ Utils backdoor
Summary: In February 2024, a malicious backdoor was introduced to the Linux utility xz within the liblzma library in versions 5.6.0 and 5.6.1 by an account using the name "Jia Tan". The backdoor gives an attacker who possesses a specific Ed448 private key remote code execution capabilities on the affected Linux system. The issue has been given the Common Vulnerabilities and Exposures number CVE-2024-3094 and has been assigned a CVSS score of 10.0, the highest possible score.
While xz is commonly present in most Linux distributions, at the time of discovery the backdoored version had not yet been widely deployed to production systems, but was present in development versions of major distributions. The backdoor was discovered by the software developer Andres Freund, who announced his findings on 29 March 2024.

Page: XZ Utils
Summary: XZ Utils (previously LZ